Constantes K y s obtenidas desde https://en.wikipedia.org/wiki/MD5

In [12]:
K = [0xd76aa478, 0xe8c7b756, 0x242070db, 0xc1bdceee, 0xf57c0faf, 
    0x4787c62a, 0xa8304613, 0xfd469501, 0x698098d8, 0x8b44f7af,
    0xffff5bb1, 0x895cd7be, 0x6b901122, 0xfd987193, 0xa679438e, 
    0x49b40821, 0xf61e2562, 0xc040b340, 0x265e5a51, 0xe9b6c7aa, 
    0xd62f105d, 0x2441453, 0xd8a1e681, 0xe7d3fbc8, 0x21e1cde6, 
    0xc33707d6, 0xf4d50d87, 0x455a14ed, 0xa9e3e905, 0xfcefa3f8, 
    0x676f02d9, 0x8d2a4c8a, 0xfffa3942, 0x8771f681, 0x6d9d6122, 
    0xfde5380c, 0xa4beea44, 0x4bdecfa9, 0xf6bb4b60, 0xbebfbc70, 
    0x289b7ec6, 0xeaa127fa, 0xd4ef3085, 0x4881d05, 0xd9d4d039,
    0xe6db99e5, 0x1fa27cf8, 0xc4ac5665, 0xf4292244, 0x432aff97,
    0xab9423a7, 0xfc93a039, 0x655b59c3, 0x8f0ccc92, 0xffeff47d, 
    0x85845dd1, 0x6fa87e4f, 0xfe2ce6e0, 0xa3014314, 0x4e0811a1, 
    0xf7537e82, 0xbd3af235, 0x2ad7d2bb, 0xeb86d391]

s = [7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22,
    5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20,
    4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23,
    6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21]

Funciones helper para la rutina principal de MD5.

In [37]:
import math

def F(B, C, D):
    return (B & C) | ((~B) & D)
 
def G(B,C,D):
    return (B & D) | (C & (~D))
 
def H(B,C,D):
    return B ^ C ^ D
 
def I(B,C,D):
    return C ^ (B | (~D))

def left_rotate(k,bits):
    bits = bits % 32
    k = k % (2 ** 32)
    result = (k<<bits) | (k>>(32 - bits))
    return result % (2**32)

def bitlen(m):
    return len(m) * 8

def pad(m):
    next_byte = 2 ** 7
    while mod(bitlen(m), 512) != 448:
        m.append(next_byte)
        next_byte = 0
    return m

def mod(n, m):
    return int(math.fmod(math.fmod(n,m)+abs(m),m))

def output(A, B, C, D):
    bytes_out = A.to_bytes(4, 'little') + B.to_bytes(4, 'little') + C.to_bytes(4, 'little') + D.to_bytes(4, 'little')
    return ''.join("{:02x}".format(b) for b in bytes_out)


Implementación de MD5 que recibe el estado inicial h0 como parámetro.

In [27]:
def custom_md5(m: str, h0: int) -> str:
    global s, K

    bytes_m = bytearray(m,'utf-8')
    
    # paso1: padding hasta ser congr 448 mod 512
    padded_m = pad(bytes_m)
    # paso2: append el largo del msje (asumiendo 64 bits)
    length_m = bitlen(m).to_bytes(8, 'little')
    padded_m.extend(length_m)
    # paso 3: Separar h0 en A, B, C, D
    bytes_h = h0.to_bytes(16, 'little')

    A0, B0, C0, D0 = int.from_bytes(bytes_h[12:], 'little'),\
                    int.from_bytes(bytes_h[8:12], 'little'),\
                    int.from_bytes(bytes_h[4:8], 'little'),\
                    int.from_bytes(bytes_h[:4], 'little')
    
    # paso 4: aplicar el algoritmo a cada bloque de 512 bits del msje
    for i in range(0, len(padded_m), 64):
        block = padded_m[i:i+64]
        A, B, C, D = A0, B0, C0, D0

        for j in range(64):
            if 0 <= j <= 15:
                f = F(B, C, D)
                g = j
            elif 16 <= j <= 31:
                f = G(B, C, D)
                g = mod((5*j + 1), 16)
            elif 32 <= j <= 47:
                f = H(B, C, D)
                g = mod((3*j + 5), 16)
            elif 48 <= j <= 63:
                f = I(B, C, D)
                g = mod((7*j), 16)

            f = f + A + K[j] + int.from_bytes(padded_m[4*g:4*(g+1)], 'little')
            A = D
            D = C
            C = B
            B = B + left_rotate(f, s[j])
        A0 += A
        B0 += B
        C0 += C
        D0 += D
        A0 %= 2 ** 32
        B0 %= 2 ** 32
        C0 %= 2 ** 32
        D0 %= 2 ** 32
    # paso 5: obtener representacion del digest segun la especificacion
    return output(A0, B0, C0, D0)
    